In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
!pip3 install flair

     |████████████████████████████████| 143kB 6.3MB/s eta 0:00:01
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=356df8d2cac4101220309f263bdb56ad98ecbc1987f034c87be04b19700c868b
  Stored in directory: /tmp/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23257 sha256=a719048d3e40260518c6b080df25f21d1162830df0182e1d9a728c1beca56cc2
  Stored in directory: /tmp/.cache/pip/wheels/15/ee/a8/6112173f1386d33eebedb3f73429cfa41a4c3084556bcee254
Successfully built sqlitedict segtok


> ## Create a Corpus

### 1) Load from simple CSV file

In [3]:
from flair.datasets import CSVClassificationCorpus

ImportError: cannot import name 'CSVClassificationCorpus'

Great development..

### 2) FastText Format

In [4]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus

In [5]:
FILE_PATH = "../input/bbctext/bbc-text.csv"
DATASET_FOLDER_PATH = "splitted_data"
MODEL_FOLDER_PATH = "model"

# file format
__label__<label_1> <text>
__label__<label_1> __label__<label_2> <text>

In [6]:
data_df = pd.read_csv(FILE_PATH).sample(frac=1)
data_df["category"] = '__label__' + data_df["category"].astype(str)

In [7]:
if not os.path.exists(DATASET_FOLDER_PATH):
    os.makedirs(DATASET_FOLDER_PATH)
data_df.iloc[0: int(len(data_df)*0.8)].to_csv(os.path.join(DATASET_FOLDER_PATH, 'train.csv'), sep='\t', index = False, header = False)
data_df.iloc[int(len(data_df)*0.8): int(len(data_df)*0.9)].to_csv(os.path.join(DATASET_FOLDER_PATH, 'dev.csv'), sep='\t', index = False, header = False)
data_df.iloc[int(len(data_df)*0.9): ].to_csv(os.path.join(DATASET_FOLDER_PATH, 'test.csv'), sep='\t', index = False, header = False);

In [8]:
corpus: Corpus = ClassificationCorpus(DATASET_FOLDER_PATH)

2019-08-07 18:08:20,211 Reading data from splitted_data
2019-08-07 18:08:20,212 Train: splitted_data/train.csv
2019-08-07 18:08:20,214 Dev: splitted_data/dev.csv
2019-08-07 18:08:20,215 Test: splitted_data/test.csv


Each line in a corpus is converted to a Sentence object annotated with the labels.

## Check distribution

In [9]:
train_df = pd.read_csv("splitted_data/train.csv", names=["label", "text"], delimiter="\t")
train_df.label.value_counts()

__label__sport            421
__label__business         412
__label__politics         333
__label__tech             315
__label__entertainment    299
Name: label, dtype: int64

In [10]:
val_df = pd.read_csv("splitted_data/dev.csv", names=["label", "text"], delimiter="\t")
val_df.label.value_counts()

__label__business         54
__label__tech             49
__label__sport            45
__label__politics         40
__label__entertainment    34
Name: label, dtype: int64

In [ ]:
test_df = pd.read_csv("splitted_data/test.csv", names=["label", "text"], delimiter="\t")
test_df.label.value_counts()

## Training a Model

In [11]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

In [12]:
if not os.path.exists(MODEL_FOLDER_PATH):
    os.makedirs(MODEL_FOLDER_PATH)

In [13]:
params_train = {
    "word_emb": 'glove',
    "flair_emb_forward": 'news-forward-fast',
    "flair_emb_backward": 'news-backward-fast',
    "hidden_size": 256,
    "reproject_words_dimension": 128
    
}

In [14]:
word_embeddings = [WordEmbeddings(params_train["word_emb"]), FlairEmbeddings(params_train["flair_emb_forward"]),
                   FlairEmbeddings(params_train["flair_emb_backward"])]

document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=params_train["hidden_size"],
                                            reproject_words=True, reproject_words_dimension=params_train["reproject_words_dimension"])

classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)

trainer = ModelTrainer(classifier, corpus)

2019-08-07 18:08:54,179 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp7ftnrlid


100%|██████████| 160000128/160000128 [00:03<00:00, 49676875.02B/s]

2019-08-07 18:08:57,560 copying /tmp/tmp7ftnrlid to cache at /tmp/.flair/embeddings/glove.gensim.vectors.npy


2019-08-07 18:08:57,791 removing temp file /tmp/tmp7ftnrlid
2019-08-07 18:08:57,888 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmp7psm6pnn


100%|██████████| 21494764/21494764 [00:00<00:00, 42176685.11B/s]

...

2019-08-07 18:09:01,389 copying /tmp/tmp5_okx8mr to cache at /tmp/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2019-08-07 18:09:01,425 removing temp file /tmp/tmp5_okx8mr


2019-08-07 18:09:05,885 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp_l9yhl8c


100%|██████████| 19689779/19689779 [00:00<00:00, 43163150.35B/s]

2019-08-07 18:09:06,526 copying /tmp/tmp_l9yhl8c to cache at /tmp/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2019-08-07 18:09:06,555 removing temp file /tmp/tmp_l9yhl8c


2019-08-07 18:09:27,069 {'entertainment', 'business', 'sport', 'politics', 'tech'}


In [ ]:
trainer.train(MODEL_FOLDER_PATH, max_epochs=10)

2019-08-07 18:09:56,381 ----------------------------------------------------------------------------------------------------
2019-08-07 18:09:56,410 Evaluation method: MICRO_F1_SCORE
2019-08-07 18:09:56,651 ----------------------------------------------------------------------------------------------------
2019-08-07 18:10:06,280 epoch 1 - iter 0/56 - loss 1.57349336
2019-08-07 18:10:25,755 epoch 1 - iter 5/56 - loss 1.60266578
2019-08-07 18:10:44,770 epoch 1 - iter 10/56 - loss 1.59392031
2019-08-07 18:11:08,488 epoch 1 - iter 15/56 - loss 1.59036079
2019-08-07 18:11:28,195 epoch 1 - iter 20/56 - loss 1.57132816
2019-08-07 18:11:45,612 epoch 1 - iter 25/56 - loss 1.55954139
2019-08-07 18:12:04,663 epoch 1 - iter 30/56 - loss 1.54682369
2019-08-07 18:12:23,018 epoch 1 - iter 35/56 - loss 1.52439698
2019-08-07 18:12:51,410 epoch 1 - iter 40/56 - loss 1.51959479
2019-08-07 18:13:09,052 epoch 1 - iter 45/56 - loss 1.51307006
2019-08-07 18:13:26,521 epoch 1 - iter 50/56 - loss 1.50106491
2

## Plot training curves

In [ ]:
from flair.visual.training_curves import Plotter

plotter = Plotter()
plotter.plot_training_curves(os.path.join(MODEL_FOLDER_PATH, 'loss.tsv'))
plotter.plot_weights(os.path.join(MODEL_FOLDER_PATH, 'weights.txt'))